In [1]:
import pandas as pd
import numpy as np 
from string import ascii_letters
import time 

In [2]:
header = ['age' ,'albumin_apache' ,'apache_2_diagnosis' ,'apache_3j_diagnosis',
 'apache_post_operative', 'arf_apache' ,'bilirubin_apache' ,'bmi',
 'bun_apache' ,'creatinine_apache', 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min' ,'d1_diasbp_max' ,'d1_diasbp_min',
 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max', 'd1_heartrate_min' ,'d1_mbp_invasive_max',
 'd1_mbp_invasive_min', 'd1_mbp_max' 'd1_mbp_min' ,'d1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min' ,'d1_resprate_max' ,'d1_resprate_min',
 'd1_spo2_max' 'd1_spo2_min' ,'d1_sysbp_invasive_max',
 'd1_sysbp_invasive_min', 'd1_sysbp_max' 'd1_sysbp_min',
 'd1_sysbp_noninvasive_max' ,'d1_sysbp_noninvasive_min', 'd1_temp_max',
 'd1_temp_min', 'elective_surgery' ,'encounter_id' ,'ethnicity', 'fio2_apache',
 'gcs_eyes_apache' ,'gcs_motor_apache', 'gcs_unable_apache',
 'gcs_verbal_apache' ,'gender' ,'glucose_apache' ,'h1_diasbp_invasive_max',
 'h1_diasbp_invasive_min', 'h1_diasbp_max' ,'h1_diasbp_min',
 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min',
 'h1_heartrate_max', 'h1_heartrate_min' ,'heart_rate_apache' ,'height',
 'hematocrit_apache' ,'hospital_admit_source' ,'hospital_death',
 'hospital_id' ,'icu_admit_source', 'icu_id' ,'icu_stay_type', 'icu_type'
 'intubated_apache' ,'map_apache' ,'paco2_apache', 'paco2_for_ph_apache'
 'pao2_apache' ,'patient_id' ,'ph_apache' ,'pre_icu_los_days',
 'readmission_status' ,'resprate_apache' ,'sodium_apache' ,'temp_apache',
 'urineoutput_apache', 'ventilated_apache' ,'wbc_apache' ,'weight']

In [4]:
header_text = "age,albumin_apache,apache_2_diagnosis,bmi,elective_surgery,encounter_id,ethnicity,gender,height,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_id,icu_stay_type,icu_type,patient_id,pre_icu_los_days,readmission_status,weight"
header = header_text.split(',')

In [7]:
results_path="Results\model_HIVAE_inputDropout_ICU_Missing20_1_z2_y5_s10_batch128\model_HIVAE_inputDropout_ICU_Missing20_1_z2_y5_s10_batch128_data_reconstruction.csv"
input_path="./ICU/test_data_preprocessed_cleaned.csv"


In [8]:
reconstructed = pd.read_csv(results_path)
input_data = pd.read_csv(input_path)

In [9]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
len(reconstructed.columns)
len(input_data.columns)
reconstructed.describe()



,56.0,3.7,37.0,21.10243942,0.0,2.0,3.0,2.0.1,180.3,4.0,0.0.1,7.0,1.0,1105.0,0.0.2,6.0,127112.0,0.104166667,0.0.3,68.6
count,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.0,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.0,4999.000000
mean,60.799059,2.675305,23.438288,28.860298,0.201040,8191.424085,2.931786,1.543309,169.395821,4.751550,0.0,107.912583,1.706141,1020.846169,0.010002,4.346469,65771.365073,0.873665,0.0,83.014010
std,16.519892,0.449323,12.604130,8.058979,0.400818,4733.287447,0.980169,0.498171,10.897444,1.714135,0.0,65.029536,0.934994,50.108080,0.132314,1.812980,38297.824500,2.464762,0.0,24.755204
min,16.000000,1.200000,1.000000,14.844926,0.000000,5.000000,1.000000,1.000000,137.200000,1.000000,0.0,1.000000,1.000000,906.000000,0.000000,0.000000,44.000000,-4.629167,0.0,38.600000
25%,51.000000,2.533000,12.000000,23.683840,0.000000,4095.000000,3.000000,1.000000,162.000000,4.000000,0.0,58.000000,1.000000,977.000000,0.000000,4.000000,32688.000000,0.070833,0.0,66.000000
50%,62.000000,2.600000,21.000000,27.372743,0.000000,8141.000000,3.000000,2.000000,170.000000,4.000000,0.0,115.000000,1.000000,1019.000000,0.000000,5.000000,65344.000000,0.196528,0.0,79.087000
75%,73.000000,2.700000,37.000000,32.360547,0.000000,12205.000000,3.000000,2.000000,177.800000,5.000000,0.0,165.000000,3.000000,1059.000000,0.000000,5.000000,99310.000000,0.465278,0.0,95.300000
max,89.000000,4.600000,44.000000,67.814990,1.000000,16504.000000,6.000000,2.000000,195.590000,11.000000,0.0,203.000000,5.000000,1111.000000,2.000000,7.000000,131050.000000,72.454167,0.0,186.000000


In [10]:
input_data.describe()

,56.0,3.7,301.0,21.10243942,0,2,3,2.1,180.3,4,0.0,7,1,1105,1.1,7.1,127112,0.104166667,0.1,68.6
count,4999.00000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.0,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000,4999.0,4999.000000
mean,58.75175,1.033775,180.917584,28.132270,0.201040,8191.424085,2.918584,1.543109,166.270404,3.532907,0.0,107.912583,1.702941,1020.846169,1.010002,5.346469,65771.365073,0.873665,0.0,81.569698
std,20.16880,1.423863,88.334976,9.321134,0.400818,4733.287447,0.999186,0.498589,25.354944,2.856664,0.0,65.029536,0.939109,50.108080,0.132314,1.812980,38297.824500,2.464762,0.0,27.216223
min,0.00000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,906.000000,1.000000,1.000000,44.000000,-4.629167,0.0,0.000000
25%,49.00000,0.000000,113.000000,23.221122,0.000000,4095.000000,3.000000,1.000000,160.000000,0.000000,0.0,58.000000,1.000000,977.000000,1.000000,5.000000,32688.000000,0.070833,0.0,64.900000
50%,62.00000,0.000000,122.000000,27.259089,0.000000,8141.000000,3.000000,2.000000,170.000000,4.000000,0.0,115.000000,1.000000,1019.000000,1.000000,6.000000,65344.000000,0.196528,0.0,79.000000
75%,73.00000,2.500000,301.000000,32.344406,0.000000,12205.000000,3.000000,2.000000,177.800000,5.000000,0.0,165.000000,3.000000,1059.000000,1.000000,6.000000,99310.000000,0.465278,0.0,95.300000
max,89.00000,4.600000,308.000000,67.814990,1.000000,16504.000000,6.000000,2.000000,195.590000,11.000000,0.0,203.000000,5.000000,1111.000000,3.000000,8.000000,131050.000000,72.454167,0.0,186.000000


In [77]:
reconstructed['hospital_death'].sum()

0.0

In [78]:
input_data['hospital_death'].sum()

0.0

In [143]:
true_data['hospital_death'].sum()


,age,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bmi,bun_apache,creatinine_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,elective_surgery,encounter_id,ethnicity,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,gender,glucose_apache,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,heart_rate_apache,height,hematocrit_apache,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_id,icu_stay_type,icu_type,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,patient_id,ph_apache,pre_icu_los_days,readmission_status,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,weight
count,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.00000,5000.000000,5000.0000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,58.75120,1.034308,23.193800,40.260600,0.223800,0.03160,0.456918,28.130864,21.156400,1.239414,24.245200,14.315800,88.645200,49.488200,87.746000,48.857600,103.857800,70.353000,34.72200,18.613000,102.924133,63.243200,101.218933,62.133800,28.262600,12.196000,98.880000,89.829600,46.006600,28.073000,147.255600,95.777400,145.370000,94.487580,35.138410,34.111681,0.201000,8189.786200,2.918600,0.152921,4.432800,6.338600,0.013600,4.973800,1.543200,142.691840,15.344000,12.603000,71.84020,58.711400,68.192400,55.571600,89.970200,81.257600,99.462400,166.273210,26.350940,3.533000,0.0,107.89240,1.702800,1020.8630,0.010000,4.34680,0.177000,85.385732,10.708620,10.708620,33.797660,65783.633200,1.869278,0.873511,0.0,23.539360,114.833600,33.150708,1178.946087,0.354000,9.743754,81.567104
std,20.16682,1.424220,12.939572,26.950472,0.416831,0.17495,1.548905,9.320732,21.694904,1.589729,38.510166,22.822082,21.438443,13.702825,23.455326,14.711091,22.891168,17.905298,59.52283,29.852113,21.870949,15.249425,25.321195,17.192446,10.665495,5.159468,7.026655,11.784886,71.719702,44.605022,27.431529,21.558497,31.759138,23.897038,8.828311,8.582126,0.400788,4734.230845,0.999086,0.293734,1.079313,1.540527,0.115835,1.635986,0.498581,99.346614,29.745793,24.596796,24.40252,20.537333,29.256811,24.337998,27.587713,25.447026,32.117585,25.353184,14.444529,2.856386,0.0,65.03869,0.939068,50.1172,0.132301,1.81295,0.381707,42.308818,19.256762,19.256762,72.362465,38303.818254,3.200863,2.464539,0.0,15.149986,51.762108,10.466330,1478.990758,0.478257,8.167493,27.214119
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [98]:
wine_reconstruction = "/Users/viljamiraiski/Skolan/MAISTERI/SGPM/Project/HI-VAE/Results/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100_data_reconstruction.csv"
wine_true = "/Users/viljamiraiski/Skolan/MAISTERI/SGPM/Project/HI-VAE/Results/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100/model_HIVAE_inputDropout_Wine_Missing20_1_z2_y5_s10_batch100_data_true.csv"
wine_data_path = "./Wine/data.csv"

In [99]:
wine_r = pd.read_csv(wine_reconstruction)
wine_t = pd.read_csv(wine_true)
wine_data = pd.read_csv(wine_data_path)

In [93]:
wine_r.describe()

,7.314,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6.0,1.0
count,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000,6496.000000
mean,7.183180,0.334550,0.316580,5.100962,0.055648,28.892397,112.025870,0.994717,3.218581,0.529563,10.461928,5.720443,0.751847
std,1.202283,0.154594,0.135522,4.552715,0.031486,16.882208,55.664648,0.002847,0.148926,0.138826,1.102098,0.832401,0.431974
min,3.800000,0.080000,0.001000,0.600000,0.009000,1.000000,6.000000,0.987130,2.740000,0.220000,8.000000,3.000000,0.000000
25%,6.500000,0.240000,0.260000,1.800000,0.039000,15.000000,77.000000,0.992400,3.120000,0.440000,9.538500,5.000000,1.000000
50%,6.986000,0.280000,0.312000,2.500000,0.048000,26.992500,113.463000,0.995000,3.200000,0.506000,10.317000,6.000000,1.000000
75%,7.500000,0.390000,0.380000,8.100000,0.066000,38.000000,151.169750,0.997000,3.310000,0.590000,11.100000,6.000000,1.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.039000,4.010000,2.000000,14.900000,9.000000,1.000000


In [101]:
wine_r.head()

,7.314,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6.0,1.0
0,6.300,0.249,0.304,1.715,0.049,26.012,112.216,0.9940,3.30,0.49,10.862,6.0,1.0
1,8.100,0.327,0.400,2.452,0.050,30.000,97.000,0.9950,3.26,0.44,10.100,5.0,1.0
2,7.200,0.230,0.320,8.500,0.058,47.000,186.000,0.9956,3.19,0.40,9.900,6.0,1.0
3,6.762,0.266,0.320,8.500,0.058,47.000,186.000,0.9960,3.19,0.40,9.900,6.0,1.0
4,8.100,0.280,0.400,2.285,0.050,30.000,97.000,0.9951,3.26,0.44,10.100,6.0,1.0


In [100]:
wine_data.head()

,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6,2
0,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,2
1,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2
2,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2
4,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2
